ဒီ Project မှာ MovieLens Dataset ကို အသုံးပြုပြီး ပိုမိုတိကျတဲ့ ခန့်မှန်းချက်တွေရဖို့ Ensemble Learning နည်းလမ်းတွေကို အသုံးပြုသွားမှာ ဖြစ်ပါတယ်။

Step 1: လိုအပ်သော Libraries များနှင့် Data များကို ခေါ်ယူခြင်း
-   Ensemble methods များဖြစ်သော RandomForest, XGBoost နှင့် VotingClassifier တို့ကို အဓိကထား အသုံးပြုပါမည်။

In [1]:
import pandas as pd

# Data Load လုပ်ခြင်း (ယခင်အဆင့်များအတိုင်း)
ratings = pd.read_csv(r'C:\Users\MSI\Desktop\Git_Project\AI_2026\Day05 Decision tree and Essemble method\example\psa_movieLens\ratings.csv')
movies = pd.read_csv(r'C:\Users\MSI\Desktop\Git_Project\AI_2026\Day05 Decision tree and Essemble method\example\psa_movieLens\movies.csv')
df = ratings.merge(movies, on='movieId')

print("Data Loaded Successfully!")

Data Loaded Successfully!


Step 2: Feature Engineering (Data ပြင်ဆင်ခြင်း)
-   Model များ တွက်ချက်နိုင်ရန် Categorical data များကို Numerical ပြောင်းလဲခြင်းနှင့် Feature အသစ်များ ထုတ်ယူခြင်း ဖြစ်ပါတယ်။

In [2]:
# Genres ကို 0/1 (One-Hot) ပြောင်းခြင်း
genres = df['genres'].str.get_dummies(sep='|')

# User behavior statistics
user_stats = df.groupby('userId')['rating'].agg(['mean', 'count', 'std']).reset_index()
user_stats.columns = ['userId', 'user_avg_rating', 'user_rating_count', 'user_rating_std']

# Movie quality statistics
movie_stats = df.groupby('movieId')['rating'].agg(['mean', 'count']).reset_index()
movie_stats.columns = ['movieId', 'movie_avg_rating', 'movie_rating_count']

# Data အားလုံးကို ပြန်ပေါင်းခြင်း
df_final = df.merge(user_stats, on='userId').merge(movie_stats, on='movieId')
df_final = pd.concat([df_final, genres], axis=1).fillna(0)

# Target Variable သတ်မှတ်ခြင်း (Rating >= 4 ဆိုလျှင် 1, မဟုတ်လျှင် 0)
df_final['high_rating'] = (df_final['rating'] >= 4).astype(int)

# Features (X) နှင့် Target (y) ခွဲခြားခြင်း
feature_cols = ['user_avg_rating', 'user_rating_count', 'user_rating_std', 
                'movie_avg_rating', 'movie_rating_count'] + list(genres.columns)
X = df_final[feature_cols]
y = df_final['high_rating']

from sklearn.model_selection import train_test_split

# Train/Test Split (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Step 3: Ensemble Method - 1: Bagging (Random Forest)
-   Bagging သည် Data ကို အပိုင်းများစွာခွဲပြီး Decision Tree အများအပြားဖြင့် မဲခွဲဆုံးဖြတ်သည့် စနစ်ဖြစ်သည်။

In [3]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Bagging (Random Forest) Model တည်ဆောက်ခြင်း
rf_model = RandomForestClassifier(n_estimators=100, max_depth=12, random_state=42)
rf_model.fit(X_train, y_train)

# ခန့်မှန်းချက်ထုတ်ခြင်း
y_pred_rf = rf_model.predict(X_test)
rf_acc = accuracy_score(y_test, y_pred_rf)

print(f"Bagging (Random Forest) Accuracy: {rf_acc:.4f}")

Bagging (Random Forest) Accuracy: 0.7546


Step 4: Ensemble Method - 2: Boosting (XGBoost)
-   Boosting သည် အမှားများကို နောက်တစ်ဆင့်တွင် ပြန်လည်ပြင်ဆင်ပြီး သင်ယူသွားသော အဆင့်မြင့်နည်းလမ်းဖြစ်သည်။

In [4]:
from xgboost import XGBClassifier 

# Boosting (XGBoost) Model တည်ဆောက်ခြင်း
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
xgb_model.fit(X_train, y_train)

# ခန့်မှန်းချက်ထုတ်ခြင်း
y_pred_xgb = xgb_model.predict(X_test)
xgb_acc = accuracy_score(y_test, y_pred_xgb)

print(f"Boosting (XGBoost) Accuracy: {xgb_acc:.4f}")

Boosting (XGBoost) Accuracy: 0.7560


Step 5: Ensemble Method - 3: Voting (Hybrid Model)
-   Voting သည် Model အများအပြား၏ အဖြေကို စုပေါင်းပြီး အများစုထောက်ခံသော အဖြေကို ယူသည့် စနစ်ဖြစ်သည်။

In [5]:
from sklearn.ensemble import VotingClassifier

# Voting Classifier တည်ဆောက်ခြင်း (RF နှင့် XGB ကို ပေါင်းစပ်ခြင်း)
voting_model = VotingClassifier(
    estimators=[('rf', rf_model), ('xgb', xgb_model)],
    voting='soft' # 'soft' သည် probability ပေါ်မူတည်၍ တွက်ချက်ခြင်းဖြစ်သည်
)
voting_model.fit(X_train, y_train)

# ခန့်မှန်းချက်ထုတ်ခြင်း
y_pred_vote = voting_model.predict(X_test)
vote_acc = accuracy_score(y_test, y_pred_vote)

print(f"Voting (Hybrid) Accuracy: {vote_acc:.4f}")

Voting (Hybrid) Accuracy: 0.7568


Step 6: Results Comparison (ရလဒ်များကို နှိုင်းယှဉ်ခြင်း)
-   ဘယ် Model က အကောင်းဆုံးလဲဆိုတာကို ဇယားဖြင့် ကြည့်ရှုပါမည်။

In [6]:
# Accuracy များကို နှိုင်းယှဉ်ခြင်း
results = pd.DataFrame({
    'Model': ['Bagging (RF)', 'Boosting (XGB)', 'Voting (Hybrid)'],
    'Accuracy Score': [rf_acc, xgb_acc, vote_acc]
}).sort_values(by='Accuracy Score', ascending=False)

print("\n--- Model Comparison Table ---")
print(results)


--- Model Comparison Table ---
             Model  Accuracy Score
2  Voting (Hybrid)        0.756788
1   Boosting (XGB)        0.756028
0     Bagging (RF)        0.754604
